In [1]:
from path import Path
len(Path('/mnt/drive1/hsun/video_datasets/CondensedMovies/videos/2019').files())

2013

In [ ]:
yt-dlp --config-location /mnt/drive1/hsun/CondensedMovies/data_prep/youtube-dl.conf -P /mnt/drive1/hsun/video_datasets/CondensedMovies/videos/2012 -a  /mnt/drive1/hsun/CondensedMovies/data/metadata/youtube-dl-dump/2012.csv

In [2]:
yt-dlp --config-location /mnt/drive1/hsun/CondensedMovies/data_prep/youtube-dl.conf -P /mnt/drive1/hsun/video_datasets/CondensedMovies/videos/2020 -a  /mnt/drive1/hsun/CondensedMovies/data/metadata/youtube-dl-dump/2020.csv

SyntaxError: invalid decimal literal (3692552401.py, line 1)

In [2]:
import cv2
import os
from path import Path
from tqdm import tqdm
from multiprocessing import Pool

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)

    # get the frames per second of the video
    fps = cap.get(cv2.CAP_PROP_FPS)

    # frame index
    idx = 0
    folder_path = '/mnt/drive1/brick1/hsun/videoSeg/data/test_folder'
    # output directory for each video
    out_dir = os.path.join(folder_path, video_path.stem)
    os.makedirs(out_dir, exist_ok=True)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # save every fps-th frame
        if idx % int(fps) == 0:
            cv2.imwrite(os.path.join(out_dir, f'frame_{int(idx / fps)}.png'), frame)
        idx += 1
    cap.release()
    
process_video(Path('/mnt/drive1/brick1/hsun/videoSeg/data/my-video.mp4'))

In [7]:


# Get the list of all videos
# video_paths = list(Path('dataset/mp4_video').files("*.mp4"))
# video_paths = [  i  for i in Path('/mnt/drive1/hsun/video_datasets/CondensedMovies/videos').dirs()]
sub_lists = [  i.files()  for i in Path('/mnt/drive1/hsun/video_datasets/CondensedMovies/videos').dirs()]

video_paths =  [item for sublist in sub_lists for item in sublist]
# Create a multiprocessing Pool
pool = Pool(processes = 100)

# Process the videos in parallel
for _ in tqdm(pool.imap_unordered(process_video, video_paths), total=len(video_paths)):
    pass

# Close the pool
pool.close()
pool.join()


100%|██████████| 29901/29901 [18:01<00:00, 27.65it/s]


In [3]:
from path import Path

In [7]:
len(Path('videos_frames').dirs())

268872

In [13]:
import os
import json
import shutil
from collections import deque
import random

def extend_frame_count(subfolder, min_video_length):
    """If a subfolder has fewer than min_video_length frames, append frames from the beginning."""
    frames = deque(sorted(os.listdir(subfolder)))
    extended_frames = list(frames)  # copy original frame order

    if len(frames)< min_video_length:
        while len(extended_frames) < min_video_length:
            extended_frames.extend(frames)

        return extended_frames
    else:
        return frames

def combine_frames(source_folder, target_folder, min_video_count, max_video_count,minimum_video_length = 10):
    """Combine frames from subfolders in source_folder into target_folder, ensuring each subfolder
    has at least min_video_length frames."""
    video_folders = deque(os.listdir(source_folder))
    total_length = video_folders
    print('video_folders length', len(video_folders))
    json_data = {}
    video_start_points = []
    frame_start_point = 0
    longvideo_count = 0
    
    while video_folders:
        longvideo_count += 1
        random_video_count = random.randint(min_video_count, max_video_count)  # Generate a random number of videos
        os.makedirs(os.path.join(target_folder, str(longvideo_count)), exist_ok=True)
        json_data = {}
        video_start_points = []
        frame_start_point = 0

        while video_folders and random_video_count > 0:
            subfolder = video_folders.popleft()
            subfolder_path = os.path.join(source_folder, subfolder)

            if os.path.isdir(subfolder_path):
                extended_frames = extend_frame_count(subfolder_path, minimum_video_length)
                video_length = len(extended_frames)
                video_start_points.append(frame_start_point)  # Store start point of each new video

                for index, frame in enumerate(extended_frames):
                    # Rename the frame to be sequential in the new folder
                    new_frame_name = f"frame_{frame_start_point + index}.jpg"
                    shutil.copy(os.path.join(subfolder_path, frame), os.path.join(target_folder, str(longvideo_count), new_frame_name))

                json_data[subfolder] = {"start_frame": frame_start_point, "end_frame": frame_start_point + video_length - 1}
                frame_start_point += video_length
                random_video_count -= 1

        print( 'left video length: ', video_folders)

        # Create a JSON file
        with open(os.path.join(target_folder, str(longvideo_count), 'frame_data.json'), 'w') as json_file:
            json.dump({"video_data": json_data, "video_start_points": video_start_points[1:]}, json_file)
        
# Define your source and target folder here:
source_folder = '/mnt/drive1/hsun/videos_frames'
target_folder = '/mnt/drive1/hsun/new_videos'
min_video_count = 2  # The minimum number of videos to consider
max_video_count = 80  # The maximum number of videos to consider
minimum_video_length = 10
# Ensure the target folder exists
os.makedirs(target_folder, exist_ok=True)

combine_frames(source_folder, target_folder, min_video_count, max_video_count,  minimum_video_length = minimum_video_length)


SyntaxError: '(' was never closed (891577297.py, line 56)

In [12]:
from path import Path

len(Path('/mnt/drive1/hsun/new_videos').dirs())

687

In [9]:
from path import Path

import shutil 

import os

files = [i.files() for i in Path('/mnt/drive1/hsun/videoSeg/data/video_datasets/CondensedMovies/video_frames').dirs()]

for i in files:
    sorted_i = sorted(i, key=lambda x: int(x.stem.split('_')[-1]))
    for j in sorted_i[-20:]:
        os.remove(j)


In [4]:
len( Path('/mnt/drive1/hsun/video_datasets/CondensedMovies/video_frames').dirs())

29901